In [1]:
import sys
import os

sys.path.append(os.getcwd())

from langchain_ollama import ChatOllama
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from app.services.rag_service import RAGService
from app.core.config import (
    LLM_PROVIDER, OLLAMA_MODEL, OLLAMA_BASE_URL, 
    OPENAI_API_KEY, OPENAI_MODEL, LLM_TEMPERATURE
)

In [3]:
print(f"Current LLM Provider: {LLM_PROVIDER}")

if LLM_PROVIDER == "openai":
    if not OPENAI_API_KEY:
        raise ValueError("OPENAI_API_KEY is required for OpenAI provider.")
    print(f"Initializing OpenAI LLM with model: {OPENAI_MODEL}")
    llm = ChatOpenAI(
        model=OPENAI_MODEL,
        temperature=LLM_TEMPERATURE,
        api_key=OPENAI_API_KEY
    )
else:
    print(f"Initializing Ollama LLM with model: {OLLAMA_MODEL}")
    llm = ChatOllama(
        model=OLLAMA_MODEL,
        temperature=LLM_TEMPERATURE, 
        base_url=OLLAMA_BASE_URL
    )

rag_service = RAGService()
print("RAG Service Initialized.")

Current LLM Provider: openai
Initializing OpenAI LLM with model: gpt-4o-mini
INFO:     2026-02-19 22:24:28,939 - app.services.rag_service - Loading Embedding Model: text-embedding-3-large...
INFO:     2026-02-19 22:24:28,954 - app.services.rag_service - Loading FAISS Vector Store from vector_store...
INFO:     2026-02-19 22:24:28,957 - app.services.rag_service - RAG Service Initialized Successfully.
RAG Service Initialized.


In [13]:
query = "سقف تعداد واحد در ترم کارآموزی چقدره؟"
print(f"Query: {query}")

messages = [HumanMessage(content=query)]
response = llm.invoke(messages)

print("\n--- Direct Chat Response ---")
print(response.content)

Query: سقف تعداد واحد در ترم کارآموزی چقدره؟

--- Direct Chat Response ---
سقف تعداد واحدهای کارآموزی در هر دانشگاه و رشته ممکن است متفاوت باشد. معمولاً در مقاطع کارشناسی، تعداد واحدهای کارآموزی بین 2 تا 6 واحد است، اما این مقدار می‌تواند بسته به سیاست‌های دانشگاه و رشته تحصیلی تغییر کند. برای اطلاعات دقیق‌تر، بهتر است به آیین‌نامه‌های آموزشی دانشگاه خود مراجعه کنید یا با مشاور تحصیلی خود تماس بگیرید.


In [14]:
query = "شرایط مشروطی در کارشناسی چیست؟"
print(f"Query: {query}\n")

docs = rag_service._retrieve_documents(query)

print(f"--- Retrieved {len(docs)} Documents ---\n")
for i, doc in enumerate(docs):
    source = doc.metadata.get("title", "Unknown Source")
    content = doc.page_content.replace("\n", " ").strip()[:200] + "..." 
    print(f"[Source {i+1}: {source}]")
    print(f"Content Preview: {content}\n")

Query: شرایط مشروطی در کارشناسی چیست؟

--- Retrieved 5 Documents ---

[Source 1: آیین‌نامه آموزشی دوره کارشناسی دانشگاه صنعتی شریف]
Content Preview: 16. درس جبرانی: درسی است که با تأیید گروه آموزشی، گذراندن آن برای رفع کمبود دانش یا مهارت دانشجو، در آغاز دوره تحصیلی، ضروری تشخیص داده می شود. این نوع درس فقط برای دانشجویانی که از رشته های غیرمرتبط ...

[Source 2: آیین‌نامه آموزشی دوره کارشناسی دانشگاه صنعتی شریف (ورودی‌های ۱۴۰۱ و ماقبل)]
Content Preview: ماده ۱۵- چنانچه میانگین نمرات دانشجو در هر نیم سال تحصیلی کمتر از ۱۲ باشد، دانشجو در آن نیم سال مشروط تلقی می شود. سقف مجاز مشروط اعم از متوالی یا متناوب در دوره کاردانی و کارشناسی ناپیوسته دو نیم سال...

[Source 3: آیین‌نامه آموزشی دوره کارشناسی دانشگاه صنعتی شریف (ورودی‌های ۱۴۰۱ و ماقبل)]
Content Preview: 15. مشروطی:  وضعیت تحصیلی است که دانشجو در یک نیم سال معدل کمتر از حد نصاب نمره تعیین شده را کسب کرده باشد. برای دوره های کاردانی و کارشناسی این حد نصاب نمره ۱۲ (از ۲۰) است. 16. رشته تحصیلی:  یکی از ش...

[Source 4: آیین‌نامه آموزشی 

In [15]:
query = "آیین‌نامه غیبت در امتحانات چیست؟"
print(f"Query: {query}\n")

system_instruction, docs = rag_service.generate_augmented_prompt(query)

if not system_instruction:
    print("No relevant documents found.")
else:
    print(system_instruction)
    print(f"\n\n--- Retrieved {len(docs)} Documents ---\n")
    for i, doc in enumerate(docs):
        source = doc.metadata.get("title", "Unknown Source")
        content = doc.page_content.replace("\n", " ").strip()[:200] + "..." 
        print(f"[Source {i+1}: {source}]")
        print(f"Content Preview: {content}\n")

    messages = [
        SystemMessage(content=system_instruction),
        HumanMessage(content=query)
    ]

    response = llm.invoke(messages)
    
    print("--- LLM Response ---")
    print(response.content)
    
    print("\n--- Sources ---")
    sources = list(set([doc.metadata.get("title", "Unknown") for doc in docs]))
    for s in sources:
        print(f"- {s}")

Query: آیین‌نامه غیبت در امتحانات چیست؟

تو هوش مصنوعی پاسخگو به سوالات آموزشی دانشگاه صنعتی شریف هستی.
                            وظیفه تو پاسخ دادن به سوالات دانشجوها *صرفاً* بر اساس متون زیر است.

                            قوانین اکید:
                            1. اگر پاسخ سوال در متن‌های زیر نیست، بگو "در قوانین موجود جوابی برای این سوال پیدا نکردم".
                            2. از خودت قانونی نساز و حدس نزن.
                            3. پاسخ را محترمانه و دقیق به زبان فارسی بده.
                            4. اگر لازم است، به نام آیین‌نامه یا قانون ارجاع بده.

                            متون قوانین (Context):
                            [Source 1: آیین‌نامه آموزشی دوره کارشناسی دانشگاه صنعتی شریف]
تبصره ۲: غیبت در امتحان پایان نیم سال منجر به نمره صفر/غایب در آن امتحان می شود و نمره نهایی درس براساس سوابق تحصیلی دانشجو در طول نیم سال در آن درس توسط مدرس قابل محاسبه و اعمال خواهد بود. در صورتی که به تشخیص شورای آموزشی دانشگاه، غیبت دانشجو موجه تشخیص داده شود، آن درس، از م